In [ ]:
import numpy as np 
import random
import pandas as pd 
from datetime import datetime, timedelta
import netCDF4 as nc
import ast
from collections import Counter

In [ ]:
def hours_since_datetime(current_datetime,target_datetime):
    time_difference = current_datetime - target_datetime
    hours_difference = time_difference.total_seconds() / 3600
    return hours_difference

#### Load ERA5 Wind data 

In [ ]:
wind_data = nc.Dataset("weather_permian.nc")
u10 = wind_data.variables["u10"][:]
v10 = wind_data.variables["v10"][:]
ws = (u10**2 + v10**2)**0.5
wind_data.close()

In [ ]:
wind_speed = ws[:,1,1]

#### Load events 

In [ ]:
events = pd.read_csv("events_with_duration_unc.csv")

In [ ]:
events.head(2)

#### Load component-scale leak rate data 

In [ ]:
leak_rates = pd.read_csv("my_leak_rate.csv")

In [ ]:
leak_rates.head(2)

In [ ]:
leaks = list(leak_rates["gpersec"])

#### Simulate emissions from unresolved events  
Based on timestamps of our simulated emission events, our reconciliation period is 6 months from 2023-11-01 to 2024-04-31

In [ ]:
# Find timestamps that within the time range of patially-resolved and resolved events 
events_time = []
events_for_sims = []
target_datetime = datetime(2023,11,1,0,0)
for _,row in events.iterrows():
    if pd.notna(row.measured_startTime):
        st = row.measured_startTime
    else:
        st = row.calculated_startTime
    st = datetime.strptime(st,'%Y-%m-%d %H:%M') 
    if pd.notna(row.measured_endTime):
        et = row.measured_endTime
    else:
        et = row.calculated_endTime
    et = datetime.strptime(et,'%Y-%m-%d %H:%M')
    hour_since_start_time = hours_since_datetime(st,target_datetime)
    hour_since_end_time = hours_since_datetime(et,target_datetime)
    times = np.arange(hour_since_start_time,hour_since_end_time+1,1)
    it = [int(t) for t in times] 
    events_time += it

In [ ]:
# non-detection time 
flyover_nodetection = [datetime(2023, 10, 24, 19, 59),
                       datetime(2023, 11, 6, 19, 52),
                       datetime(2023, 11, 22, 19, 1),
                       datetime(2023, 11, 22, 19, 2),
                       datetime(2023, 12, 16, 18, 3),
                       datetime(2023, 12, 16, 18, 4),
                       datetime(2024, 2, 18, 20, 22),
                       datetime(2024, 2, 18, 20, 23),
                       datetime(2024, 3, 1, 19, 21),
                       datetime(2024, 3, 1, 19, 22)]

In [ ]:
# flyover passes 
passes = [1, 1, 2, 3, 3, 2, 2, 2, 3, 3, 3, 1, 1]

In [ ]:
Durations = [] 
for _,row in events.iterrows():
    if pd.notna(row.measured_duration):
        dur = row.measured_duration
        Durations.append(dur)

In [ ]:
# calcualte frequency 
frequency_distribution = Counter(Durations)
total_count = len(Durations)
# Calculate the probability distribution
probability_distribution = {value: frequency / total_count for value, frequency in frequency_distribution.items()}

In [ ]:
# Extract values and their probabilities
values = list(probability_distribution.keys())
probabilities = list(probability_distribution.values())

In [ ]:
mc = 0
MC = 10000 
EMs = [] 
while mc < MC:
    E_miss = 0 
    target_datetime = datetime(2023,11,1,0,0,0)
    init_datetime = datetime(2023,11,1,0,0,0)
    while init_datetime < datetime(2024,5,1,0,0,0): 
        Q_sample = random.sample(leaks,1)[0]
        Q_sample = (Q_sample * 3600)/1000 
        Q_sample = Q_sample * random.randint(1, 5440)
        hour_since_start_time = hours_since_datetime(init_datetime,target_datetime)

        if init_datetime not in flyover_nodetection:
            # randomly sample a kairos flyover pass 
            N = random.sample(passes,1)[0]
            detection_label = False 
            for n in range(N): 
                # get random u (wind speed) - wind speed sample from ERA5  
                u = np.random.choice(wind_speed, size = 1)
                POD = 1 - (1+(((0.00771)/u**1.41)*Q_sample **1.87)**2)**-1.5
                rdp = random.random()
                if POD > rdp: 
                    # detection success 
                    detection_label = True
                    break
             
            if not detection_label and hour_since_start_time not in events_time: 
                N = 7 # number of qube sensors installed on site 
                detection_label = False 
                for n in range(N): 
                    # get random u (wind speed) - wind speed sample from ERA5 
                    u = np.random.choice(wind_speed, size = 1)
                    normQ = Q_sample /u 
                    POD_cms = 1/(1 + np.exp(-0.309)+np.exp(-1.047*normQ))
                    rdp = random.random()
                    if POD_cms > rdp: 
                        # detection success 
                        detection_label = True
                        break
                        
            if not detection_label:
                sampled_duration = random.choices(values, weights=probabilities, k=1)[0]
                E_miss += Q_sample * sampled_duration
                init_datetime += timedelta(hours=int(sampled_duration))
            else:
                init_datetime += timedelta(hours=1)
        else: 
            init_datetime += timedelta(hours=1)
        
    if (mc % 100) == 0: 
        print(mc)
    EMs.append(E_miss)
    mc += 1 
    

In [ ]:
len(EMs)

In [ ]:
np.mean(EMS), np.std(EMs)

In [ ]:
UEEs = {"Emissions_from_unresovled_event":EMs}

In [ ]:
df = pd.DataFrame(UEEs)

In [ ]:
df.head(2)

In [ ]:
df.to_csv("Emissions_from_unresolved_event.csv")